Maxime Marchand
# Astrophysics and Data Science : Project 2
## Extracting individual spectra of spectroscopic binaries using the Fourier transform

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy import constants as const
from astropy import units as u
from tqdm import tqdm

### Part 2 : Data loading and visualization

In [ ]:
# Reading data file
fName = 'data.npz'
data  = dict(np.load(fName))

# Changing key names
data['la']   = data.pop('la_nm')      # Wavelengths
data['vA']   = data.pop('vA_m_s')     # Radial velocity of A
data['vB']   = data.pop('vB_m_s')     # Radial velocity of B
data['spec'] = data.pop('spectra')    # Spectra of the binary (axis 0 = time, axis 1 = wavelength)

# Plotting data
xmin = 587
xmax = 592

fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(16, 4), dpi=300)

# Selection of the points between xmin and xmax. The * operator acts as an AND statement
sel = (data['la'] >= xmin) * (data['la'] <= xmax)

ax[0].plot(data['la'], data['spec'][0], lw=0.5)
ax[0].plot(data['la'][sel], data['spec'][0][sel], c='k')
ax[0].axvline(xmin, c='grey')
ax[0].axvline(xmax, c='grey')

ax[1].set_xlabel("Wavelength [nm]")
ax[1].set_ylabel("Spectra")
ax[1].set_xlim(xmin, xmax)
ax[1].plot(data['la'], data['spec'][0], c='k');

### Part 3 : Doppler shift of the spectra

>##### Point a)
Give the expression of $F$ as a function of $F_A$, $F_B$, $\Delta\lambda_A$, and $\Delta\lambda_B$.

The difference in wavelength due to the motion of a source can be written as :

\begin{equation}
\Delta\lambda = \lambda_{ref} \left( \sqrt{\frac{1+\beta}{1-\beta}} - 1 \right)
\end{equation}

Where, in our case, $\lambda_{ref} = 589.5$ nm is supposed to be constant. With this definition, one can rewrite the expression of $F(\lambda)$ as a function of $F_A(\lambda)$, $F_B(\lambda)$, $\Delta\lambda_A$ and $\Delta\lambda_B$ :

\begin{equation}
F(\lambda) = F_A(\lambda - \Delta\lambda_A) + F_B(\lambda - \Delta\lambda_B)
\end{equation}

>##### Point b)
Compute the values of $\Delta\lambda_A$ and $\Delta\lambda_B$ for each spectrum.

In [ ]:
def compute_delta_lambda(nu, lambda_ref):
    beta = nu / const.c.value
    return lambda_ref * ( np.sqrt( (1+beta)/(1-beta) ) - 1 )

lambda_ref = 589.5

delta_lambdas = np.ndarray(shape=(len(data['spec']), 2))

for i in range(len(data['spec'])):
    vA = data['vA'][i]
    vB = data['vB'][i]
    deltaLambdaA = compute_delta_lambda(vA, lambda_ref)
    deltaLambdaB = compute_delta_lambda(vB, lambda_ref)
    delta_lambdas[i, :] = [deltaLambdaA, deltaLambdaB]

Lets make a graph of the values of $\Delta\lambda_{A, B}$ for each measurment :

In [ ]:
x = np.arange(0, len(data['spec']))

fig, ax = plt.subplots(figsize=(16, 2), dpi=300)
ax.set_xlabel("Measurment")
ax.set_ylabel("$\Delta\lambda$")

ax.plot(x, delta_lambdas[:, 0], marker='o', label="$\Delta\lambda_A$")
ax.plot(x, delta_lambdas[:, 1], marker='o', label="$\Delta\lambda_B$")
ax.axhline(0, c='k', alpha=0.3);

ax.legend();

As we can observe the points always have opposite values, which confirms the rotation of the two stars, one being redshiftef as the second one is blueshifted, and vice versa. The blue line is more flat, which could be explained by the fact that the star is more massive.

### Part 4 : Doppler shifts in Fourier space

>##### Point a)
For a given spectrum $F(\lambda)$, give the expression of its Fourier transform $\hat{F}(\nu)$ as a function of the Fourier transforms of both components ($\hat{F}_A$ and $\hat{F}_B$).

We saw in the point 3a that the spectrum of the system $F(\lambda)$ can be written as :

\begin{equation}
F(\lambda) = F_A(\lambda + \Delta\lambda_A) + F_B(\lambda + \Delta\lambda_B)
\end{equation}

Let recall the Fourier transform of a function $f(\lambda)$ :

\begin{equation}
\hat{f}(\nu) = \frac{1}{\sqrt{2\pi}} \int_{-\infty}^{\infty} f(\lambda) e^{-2\pi i \nu\lambda} d\lambda
\end{equation}

Let compute :


\begin{eqnarray}
\hat{F}(\nu) & = & \frac{1}{\sqrt{2\pi}} \int_{-\infty}^{\infty} F(\lambda) e^{-2\pi i \lambda \nu} \: d\lambda
               =   \frac{1}{\sqrt{2\pi}} \int_{-\infty}^{\infty} \big[ F_A(\lambda - \Delta\lambda_A) 
                    + F_B(\lambda - \Delta\lambda_B) \big] e^{-2\pi i \lambda \nu} \;d\lambda \\
             & = & \frac{1}{\sqrt{2\pi}} \int_{-\infty}^{\infty} F_A(\lambda - \Delta\lambda_A) e^{-2\pi i \lambda \nu} \; d\lambda + \frac{1}{\sqrt{2\pi}} \int_{-\infty}^{\infty} F_B(\lambda - \Delta\lambda_B) e^{-2\pi i \lambda \nu} \; d\lambda \\
             & = & \frac{1}{\sqrt{2\pi}} \int_{-\infty}^{\infty} F_A(\lambda') e^{-2\pi i \nu (\lambda' + \Delta\lambda_A)} \; d\lambda' + \frac{1}{\sqrt{2\pi}} \int_{-\infty}^{\infty} F_B(\lambda'') e^{-2\pi i \nu (\lambda''+\Delta\lambda_B) } \; d\lambda'' \\
             & = & e^{-2\pi i \nu \Delta\lambda_A} \hat{F}_A(\nu) + e^{-2\pi i \nu \Delta\lambda_B} \hat{F}_B(\nu)
\end{eqnarray}


We finally find :

\begin{equation}
\hat{F}(\nu) = e^{-2\pi i\nu \Delta\lambda_A} \hat{F}_A(\nu) + e^{-2\pi i\nu \Delta\lambda_B} \hat{F}_B(\nu)
\end{equation}

>##### Point b)
Since we have 34 spectra taken at different epochs, we have 34 different Fourier transforms $\hat{F}_k(\nu)$ corresponding to different shifts $\Delta\lambda_{A, k}$, $\Delta\lambda_{B, k}$. We now want to solve for the values of $\hat{F}_A(\nu)$ and $\hat{F}_B(\nu)$. Show that $\hat{F}_A(\nu)$ and $\hat{F}_B(\nu)$ can be solved for, at each frequency $\nu$ independently.

We computed in the previous point the Fourier transform of the spectra $\hat{F}(\nu)$. As we can observe, for each measurement, the difference in wavelength $\Delta\lambda_{A,B}$ only appears in the phases (complex exponentials). Thus, for each value of the frequency $\nu$, the Fourier transform of the two objects $\hat{F}_A(\nu)$ and $\hat{F}_B(\nu)$ are independant of the value of the redshift/blueshift.

>##### Point c)
Show that for a given frequency $\nu$, solving for $\hat{F}_A(\nu)$ and $\hat{F}_B(\nu)$ requires to solve a linear problem of the form $y = a\theta + \varepsilon$. What does play the role of $y, A, \theta$, and $\varepsilon$ here ?

Let $N$ be the number of measurements, and $i\in[1, N]$ the index of a measurement. Considering the previous equation, one can write a system of equations in the form $y = A\theta + \varepsilon$ :

\begin{equation}
\begin{bmatrix} \hat{F}_1(\nu) \\ \vdots \\ \hat{F}_N(\nu) \end{bmatrix} = 
\begin{bmatrix} e^{-2\pi i \nu \Delta\lambda_{A1}} & e^{-2 \pi i \nu \Delta\lambda_{B1}} \\ \vdots & \vdots \\ e^{-2\pi i \nu \Delta\lambda_{AN}} & e^{-2 \pi i \nu \Delta\lambda_{BN}} \end{bmatrix}
\begin{bmatrix}
\hat{F}_A(\nu) \\ \\ \hat{F}_B(\nu)
\end{bmatrix} + 
\begin{bmatrix}
\varepsilon_1 \\ \vdots \\ \varepsilon_N
\end{bmatrix}
\end{equation}

Here, $y$ plays the role of the spectrum to be modelled, $A$ the basis matrix of the linear system, $\theta$ the parameters of our model, and $\varepsilon$ the noise due to the fact that the measurements are subject to noise. :-)


### Part 5 : Side effects and apodization
>##### Point a)
Compute the 2D array of apodized spectra

The apodized spectra is given by :

\begin{equation}
F_{apo}(\lambda_i) = \sin^2\left(\frac{\pi k}{n}\right) (F_{\lambda_i} - C)
\end{equation}

In [ ]:
C = 54440

# We keep the apodized spectra in a different variable to keep the original data
apodized_spec = data['spec'].copy()

num_pixels = len(data['la'])
k = np.arange(0, len(apodized_spec[0]), 1)   # k = [0, 1, 2, 3, ..., numPixels-1]


for i in range(len(apodized_spec)):
    apodized_spec[i] = np.sin(np.pi * k / num_pixels)**2 * (apodized_spec[i] - C)
    

# Plotting the data
fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(16, 4), dpi=300)
fig.subplots_adjust(hspace=0)
ax[0].set_title("[above] Original spectra - [below] Apodized spectra")
ax[0].plot(data['la'], data['spec'][0], c='k')
ax[1].plot(data['la'], apodized_spec[0], c='k');

ax[0].axhline(np.max(data['spec'][0]), c='r')
ax[1].axhline(0, c='r');

>##### Point b) 
Compute the 2D array of their Fourier transforms (using np.fft)

In [ ]:
apodized_spec_FFT = np.fft.rfft(apodized_spec)

>##### Point c)
Loop over the frequency $\nu \neq 0$ and solve for $\hat{F}_A(\nu)$ anf $\hat{F}_B(\nu)$

In [ ]:
dlambda = data['la'][1] - data['la'][0]
freq    = np.fft.rfftfreq(len(data['la']), d=dlambda)

epsilons = np.ones(len(data['spec'])) # The epsilon can be different for each measurment, but it is the same for each freq.

F_A_FFT = np.ndarray(shape=len(freq), dtype=complex)
F_B_FFT = np.ndarray(shape=len(freq), dtype=complex)

for i in range(1, len(freq)):
    nu = freq[i]
    X  = np.exp(-2 * np.pi * 1j * nu * delta_lambdas) # Basis matrix
    Sigma      = np.diag(epsilons**2)
    Weight     = np.linalg.inv(Sigma)
    Ca         = np.linalg.inv(X.T @ Weight @ X)
    aMLE       = Ca @ X.T @ Weight @ apodized_spec_FFT[:, i]
    F_A_FFT[i] = aMLE[0]
    F_B_FFT[i] = aMLE[1]

>##### Point d)
>What happens when $\nu = 0$ ? Comment on the reason and consequences.

If $\nu = 0$, then the wavelength $\lambda$ is infinite, which is not physically consistent. Furthermore, if $\nu = 0$, the basis matrix is full of ones, thus the covariance matrix $C_\theta = X^T \Sigma^{-1} X$ cannot be inverted.

>##### Point e)
>Compute the individual apodized spectra of the two stars from their Fourier transforms

In [ ]:
Fapo_A = np.fft.irfft(F_A_FFT)
Fapo_B = np.fft.irfft(F_B_FFT)

>##### Point f)
>Finally correct the individual spectra from the apodization factor. Comment your results.

In [ ]:
F_A = Fapo_A / (np.sin(np.pi * k / num_pixels)**2) + C
F_B = Fapo_B / (np.sin(np.pi * k / num_pixels)**2) + C

Let have a look at the individual spectra :

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(16, 4), dpi=300)
fig.subplots_adjust(hspace=0)

lim = 15000

ax[0].plot(data['la'][lim:-lim], F_A[lim:-lim], c='k')
ax[1].plot(data['la'][lim:-lim], F_B[lim:-lim], c='k');